In [1]:
# 1. Import necessary libraries
import os
import numpy as np
import qcodes as qc
from qcodes.dataset import (
    Measurement,
    do1d,
    initialise_or_create_database_at,
    load_or_create_experiment,
)
from qcodes.station import Station
from qcodes.tests.instrument_mocks import (
    DummyInstrument,
    DummyInstrumentWithMeasurement,
)
import IPython.lib.backgroundjobs as bg
from plottr.apps import inspectr

# 2. Setting up the configuration
qc.logger.start_all_logging()

# 3. Initializing instruments
dac = DummyInstrument('dac', gates=['ch1', 'ch2'])
dmm = DummyInstrumentWithMeasurement(name='dmm', setter_instr=dac)
station = qc.Station(dmm, dac)

# 4. Setting up a database
db_file_path = os.path.join(os.getcwd(), 'plottr_for_live_plotting_tutorial.db')
initialise_or_create_database_at(db_file_path)
exp = load_or_create_experiment(
    experiment_name='plottr_for_live_plotting_with_subsecond_refresh_rate',
    sample_name="no sample"
)

# 5. Setting up measurements
meas = Measurement(exp=exp)
meas.register_parameter(dac.ch1)
meas.register_parameter(dmm.v1, setpoints=(dac.ch1,))
meas.write_period = 0.1

# 6. Running the measurements
with meas.run() as datasaver:
    for set_v in np.linspace(0, 25, 100):
        dac.ch1.set(set_v)
        # get_v = dmm.v1.get()
        get_v = np.random.rand(2, size=10)
        datasaver.add_result((dac.ch1, set_v), (dmm.v1, get_v))

    dataset = datasaver.dataset

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\Wister\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\Wister\.qcodes\logs\230822-34836-qcodes.log
Starting experimental run with id: 9. 


In [43]:
# 5. Setting up measurements
meas = Measurement(exp=exp)
meas.register_parameter(dac.ch1)
meas.register_parameter(dac.ch2)
meas.register_parameter(dmm.v1, setpoints=(dac.ch1,dac.ch2))
meas.register_parameter(dmm.v2, setpoints=(dac.ch1,dac.ch2))
meas.write_period = 0.1

# 6. Running the measurements
with meas.run() as datasaver:
    for set_v2 in np.linspace(0, 100, 100):
        for set_v in np.linspace(0, 25, 100):
            dac.ch1(set_v)
            dac.ch2(set_v2)
            get_v2 = dmm.v2.get()
            data_array = np.random.randint(5, size=20)
            # data_array = set_v+5
            datasaver.add_result((dac.ch1, set_v),(dac.ch2, set_v2), (dmm.v1, data_array), (dmm.v2, get_v2))

    dataset = datasaver.dataset

Starting experimental run with id: 23. 


In [23]:
get_v = dmm.v2
dmm.print_readable_snapshot()

dmm:
	parameter value
--------------------------------------------------------------------------------
IDN :	{'vendor': 'QCoDeS', 'model': "<class 'qcodes.tests.instrument_mocks.Du...
v1  :	5.0859 (V)
v2  :	0.00053588 (V)
